In [5]:
!pip install numpy

In [ ]:
import os
import numpy as np
from scipy.cluster.hierarchy import dendrogram, to_tree
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

- Nine IoT devices were used.
- The devices were 2 smart doorbells, 1 smart thermostat, 1 smart babymonitor, 4 security cameras and 1 webcam.
- Traffic was captured when the devices were in normal execution and after infection with malware.
- Mirai and BashLite (aka gafgyt) malware were used.
- From the network traffic, 115 features were extracted as described in [1].

In [2]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/kaggle/input/nbaiot-dataset", filename),
        delimiter=",",
        skip_header=1
    )
benign = load_nbaiot("1.benign.csv")
X_train = benign[:40000]
X_test0 = benign[40000:]
X_test1 = load_nbaiot("1.mirai.scan.csv")
X_test2 = load_nbaiot("1.mirai.ack.csv")
X_test3 = load_nbaiot("1.mirai.syn.csv")
X_test4 = load_nbaiot("1.mirai.udp.csv")
X_test5 = load_nbaiot("1.mirai.udpplain.csv")

In [3]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape)

(40000, 115) (9548, 115) (107685, 115) (102195, 115) (122573, 115) (237665, 115) (81982, 115)


In [4]:
def agglomerative_clustering(data):
    # sqrt makes this a proper distance metric
    correlation_distance = np.sqrt(1-np.corrcoef(data.T))
    ac = AgglomerativeClustering(
        n_clusters=None,
        affinity="precomputed",
        linkage="single",
        distance_threshold=0
    )
    ac.fit(correlation_distance)
    return ac

feature_mapping_phase = 7777
ac = agglomerative_clustering(X_train[:feature_mapping_phase])

In [5]:
def linkage_matrix(model):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    return np.column_stack([model.children_, model.distances_, counts]).astype(float)

lm = linkage_matrix(ac)

In [6]:
import matplotlib.pyplot as plt

dendrogram(lm)
plt.close("all")

In [7]:
def find_subsets(tree, max_cluster_size=10):
    if tree.count <= max_cluster_size:
        return [np.array(tree.pre_order())]
    recursion1 = find_subsets(tree.get_left(), max_cluster_size)
    recursion2 = find_subsets(tree.get_right(), max_cluster_size)
    return recursion1+recursion2
    
subsets = find_subsets(to_tree(lm))

subsets

[array([55, 62]),
 array([78, 75, 72, 66, 69]),
 array([14, 29, 11, 26,  8, 23,  2, 17,  5, 20]),
 array([48, 34, 41]),
 array([79, 76, 73, 67, 70]),
 array([50, 36, 43]),
 array([57]),
 array([101,  44,  71,  30,  65,  37,  68,  94,  80,  87]),
 array([ 64, 108,  51,  74,  58,  77,  53,  60]),
 array([46]),
 array([32, 39]),
 array([114]),
 array([63]),
 array([56]),
 array([110,  96, 103,  82,  89]),
 array([ 86,  35,  85,  93,  42,  92,  49,  99, 100]),
 array([ 84,  91,  98, 105, 112, 107, 106, 113]),
 array([47, 33, 40, 54, 61]),
 array([59]),
 array([52]),
 array([ 83,  90,  97, 104, 111]),
 array([ 7, 22]),
 array([ 1, 16,  4, 19]),
 array([ 45,  31,  38, 109,  81,  88,  95, 102]),
 array([10, 25, 13, 28]),
 array([12, 27,  9, 24,  6, 21,  0, 15,  3, 18])]

In [8]:
class Autoencoder(Model):
    def __init__(self, n):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(n, activation="relu"),
            layers.Dense(int(0.75*n), activation="relu"),
        ])
        self.decoder = layers.Dense(n, activation="relu")
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

def compile_and_train(ae, x):
    ae.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    ae.fit(
        x=x,
        y=x,
        # in reality, it is supposed to be an online algorithm, so
        # we make only 1 pass over the training data
        epochs=1
    )

class Ensemble:
    def __init__(self, feature_subsets):
        self.map = feature_subsets
        self.scaler_ensemble = MinMaxScaler()
        self.scaler_output = MinMaxScaler()
        self.ensemble_layer = []
        for subset in feature_subsets:
            ae = Autoencoder(len(subset))
            self.ensemble_layer += [ae]
        self.output_layer = Autoencoder(len(feature_subsets))
        
    def train(self, data):
        scaled = self.scaler_ensemble.fit_transform(data)
        loss_ensemble = []
        
        for i, (features, ae) in enumerate(zip(self.map, self.ensemble_layer)):
            x = scaled[:, features]
            print(f"##**~~__ Autoencoder {i+1}/{len(self.map)} for {len(features)} dimensions")
            compile_and_train(ae, x)
            loss_ensemble += [losses.mse(x, ae(x))]
            
        # Because of the above loop, loss_ensemble now has shape
        # (n_autoencoders, n_samples). But for the output layer, the previous
        # layer outputs are actually treated as features. Therefore transpose
        loss_ensemble = self.scaler_output.fit_transform(np.array(loss_ensemble).T)
        print(f"##**~~__ Output Autoencoder for {loss_ensemble.shape[1]} dimensions")
        compile_and_train(self.output_layer, loss_ensemble)
        loss_out = losses.mse(loss_ensemble, self.output_layer(loss_ensemble))
        self.threshold = np.mean(loss_out)+np.std(loss_out)
    
    def predict(self, data):
        scaled = self.scaler_ensemble.transform(data)
        loss_ensemble = []
        
        for features, ae in zip(self.map, self.ensemble_layer):
            x = scaled[:, features]
            loss_ensemble += [losses.mse(x, ae(x))]
            
        loss_ensemble = self.scaler_output.transform(np.array(loss_ensemble).T)
        loss_out = losses.mse(loss_ensemble, self.output_layer(loss_ensemble))
        return loss_out > self.threshold

In [9]:
ensemble = Ensemble(subsets)
ensemble.train(X_train[feature_mapping_phase:])

##**~~__ Autoencoder 1/26 for 2 dimensions
1007/1007 [==============================] - 1s 763us/step - loss: 3.9535e-04
##**~~__ Autoencoder 2/26 for 5 dimensions
1007/1007 [==============================] - 1s 769us/step - loss: 8.1684e-04
##**~~__ Autoencoder 3/26 for 10 dimensions
1007/1007 [==============================] - 1s 780us/step - loss: 3.0822e-04
##**~~__ Autoencoder 4/26 for 3 dimensions
1007/1007 [==============================] - 1s 779us/step - loss: 9.1522e-05
##**~~__ Autoencoder 5/26 for 5 dimensions
1007/1007 [==============================] - 1s 817us/step - loss: 0.0012
##**~~__ Autoencoder 6/26 for 3 dimensions
1007/1007 [==============================] - 1s 786us/step - loss: 0.2127
##**~~__ Autoencoder 7/26 for 1 dimensions
1007/1007 [==============================] - 1s 730us/step - loss: 0.2534
##**~~__ Autoencoder 8/26 for 10 dimensions
1007/1007 [==============================] - 1s 838us/step - loss: 0.0015
##**~~__ Autoencoder 9/26 for 8 dimensions
100

In [10]:
test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5]

for i, x in enumerate(test_data):
    print(i)
    print(f"Shape of data: {x.shape}")
    outcome = ensemble.predict(x)
    print(f"Detected anomalies: {np.mean(outcome)*100}%")
    print()

0
Shape of data: (9548, 115)
Detected anomalies: 0.6388772517804776%

1
Shape of data: (107685, 115)
Detected anomalies: 100.0%

2
Shape of data: (102195, 115)
Detected anomalies: 100.0%

3
Shape of data: (122573, 115)
Detected anomalies: 100.0%

4
Shape of data: (237665, 115)
Detected anomalies: 100.0%

5
Shape of data: (81982, 115)
Detected anomalies: 100.0%

